# Bead on a hoop

A circular wire hoop rotates with constant angular velocity $\omega$ about a vertical diameter. A small bead
moves, without friction, along the hoop.

The equation of motion can be shown to be (using the standard notation in classical mechanics) :

$$ \ddot{\theta} = -\omega_c^2 \sin \theta + \omega^2 \sin \theta \, \cos \theta $$

with $\omega_c = \sqrt{g/R}$, where the gravity acceleration is denoted by g and the radius of the hoop is denoted R. 

Let $q=\theta$ and $p = \dot{\theta}$. Then, we can switch to a first order system of differential equations :

$$
\left\{\begin{aligned}
\dot{q} & = p\\
\dot{p} & = \sin q \, (\omega^2 \cos q - \omega_c^2)
\end{aligned}\right.
$$

In [ ]:
import mylib.integration as integration
from mylib.model import bead_hoop_model

import numpy as np

from bokeh.io import output_notebook
from bokeh.plotting import show, figure
from bokeh.layouts import column

output_notebook(hide_banner=True)

### Symplectic schemes

In [ ]:
def plot_sol():

    qini = 1.
    pini = 1.
    yini = (qini, pini)

    tini = 0.
    tend = 10
    nt = 1001    

    bhm = bead_hoop_model(omega=1.)
    H_p = bhm.H_p
    H_q = bhm.H_q
    fcn = bhm.fcn

    fig = figure(plot_height=500, plot_width=950, title="Phase space (click on legend to hide corresponding graph)")

    # forward Euler integration
    sol_fe = integration.forward_euler(tini, tend, nt, yini, fcn)
    plt_fe = fig.x(sol_fe.y[0], sol_fe.y[1], legend="forward euler")

    # backward Euler integration
    sol_be = integration.backward_euler(tini, tend, nt, yini, fcn)
    plt_be = fig.x(sol_be.y[0], sol_be.y[1], color="goldenrod", legend="backward Euler")
    
    # symplectic euler integration
    sol_se = integration.symplectic_euler(tini, tend, nt, yini, H_p, H_q)
    plt_se = fig.x(sol_se.y[0], sol_se.y[1], color="crimson", legend="symplectic Euler")
    
    #  Stormer-Verlet integration
    sol_sv = integration.stormer_verlet(tini, tend, nt, yini, H_p, H_q)
    plt_sv = fig.x(sol_sv.y[0], sol_sv.y[1], color="purple", legend="Stormer-Verlet")

    # optimized order 8 step 15 integration
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    plt_o815 = fig.x(sol_o815.y[0], sol_o815.y[1], color="green", legend="optimized order 8 step 15")
    
    fig.legend.background_fill_alpha = 0.75
    fig.legend.click_policy="hide"
    show(fig)
    
plot_sol()

### Comparison between symplectic schemes

In [ ]:
def compare_symplectic():
     
    qini = 1.
    pini = 1.
    yini = (qini, pini)

    tini = 0.
    tend = 5
    nt = 51
    omega = 1.
    
    bhm = bead_hoop_model(omega=omega)
    H_p = bhm.H_p
    H_q = bhm.H_q
    fcn = bhm.fcn

    fig = figure(plot_height=500, plot_width=950, title="Phase space")

    # symplectic euler integration
    sol_se = integration.symplectic_euler(tini, tend, nt, yini, H_p, H_q)
    plt_se = fig.x(sol_se.y[0], sol_se.y[1], color="crimson", legend="symplectic Euler")

    #  Stormer-Verlet integration
    sol_sv = integration.stormer_verlet(tini, tend, nt, yini, H_p, H_q)
    plt_sv = fig.x(sol_sv.y[0], sol_sv.y[1], color="purple", legend="Stormer-Verlet")

    # optimized order 8 step 15 integration
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    plt_o815 = fig.x(sol_o815.y[0], sol_o815.y[1], color="green", legend="optimized order 8 step 15")
    
    fig.legend.background_fill_alpha = 0.75
    fig.legend.click_policy="hide"
    show(fig)
    
compare_symplectic()

In [ ]:
def show_invariant():

    qini = 1.
    pini = 1.
    yini = (qini, pini)
    omega = 1.

    tini = 0.
    tend = 50
    nt = 501
    
    bhm = bead_hoop_model(omega=omega)
    H_p = bhm.H_p
    H_q = bhm.H_q
    fcn = bhm.fcn
    hamiltonian = bhm.hamiltonian

    fig_ham = figure(x_range=(tini, tend), plot_height=500, plot_width=950, title="Hamiltonian")
    t = np.linspace(tini, tend, nt)
    
    ham_ini = hamiltonian(np.array(yini))

    # symplectic euler integration
    sol_se = integration.symplectic_euler(tini, tend, nt, yini, H_p, H_q)
    ham_se = hamiltonian(sol_se.y)
    plt_ham_se = fig_ham.x(sol_se.t, ham_ini - ham_se[0], color="crimson", legend="symplectic Euler")
    
    # Stormer-Verlet integration
    sol_sv = integration.stormer_verlet(tini, tend, nt, yini, H_p, H_q)
    ham_sv = hamiltonian(sol_sv.y)
    plt_ham_sv = fig_ham.x(sol_sv.t, ham_ini - ham_sv[0], color="blue", legend="Stormer-Verlet")

    # optimized order 8 step 15 integration
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    ham_o815 = hamiltonian(sol_o815.y)
    plt_ham_o815 = fig_ham.x(sol_o815.t, ham_ini - ham_o815[0], color="green", legend="optimized order 8 step 15")

    fig_ham.legend.background_fill_alpha = 0.75
    fig_ham.legend.click_policy="hide"
    fig_ham.legend.location = "bottom_left"
    show(fig_ham) 

show_invariant()

### Composition scheme optimized 8-15 vs Dopri853

In [ ]:
def show_eight_order_sol():

    qini = 1.
    pini = 1.
    yini = (qini, pini)
    omega = 1.

    tini = 0.
    tend = 80
    nt = 4001
    
    bhm = bead_hoop_model(omega=omega)
    H_p = bhm.H_p
    H_q = bhm.H_q
    fcn = bhm.fcn
    hamiltonian = bhm.hamiltonian
    
    fig_sol = figure(plot_height=500, plot_width=950, title="Phase space (click on legend to hide corresponding graph)")
    fig_ham = figure(x_range=(tini, tend), plot_height=500, plot_width=950, title="Hamiltonian")
    
    ham_ini = hamiltonian(np.array(yini))

    # optimized order 8 step 15 integration
    sol_o815 = integration.optimized_815(tini, tend, nt, yini, H_p, H_q)
    fig_sol.x(sol_o815.y[0], sol_o815.y[1], color="green", legend="optimized order 8 step 15")
    ham_o815 = hamiltonian(sol_o815.y)
    plt_ham_o815 = fig_ham.x(sol_o815.t, ham_ini - ham_o815[0], color="green", legend="optimized order 8 step 15")
    
    # dopri853
    sol_dopri853 = integration.dopri853(tini, tend, nt, yini, fcn, tol=1.e-6)
    fig_sol.x(sol_dopri853.y[0], sol_dopri853.y[1], color="crimson", legend="dopri853")
    ham_dopri853 = hamiltonian(sol_dopri853.y)
    fig_ham.x(sol_dopri853.t, ham_ini - ham_dopri853[0], color="crimson", legend="dopri853")

    fig_sol.legend.background_fill_alpha = 0.75
    fig_sol.legend.click_policy="hide"
    fig_ham.legend.background_fill_alpha = 0.75
    fig_ham.legend.click_policy="hide"
    fig_ham.legend.location = "top_left"
    show(column(fig_sol,fig_ham))
    
show_eight_order_sol()